# $\color{purple}{\text{Understanding Missing Data and How to Deal with It (Part 2)}}$

### $\color{purple}{\text{Colab Environmental Setup}}$

### $\color{purple}{\text{Libraries for this lesson}}$

In [1]:
import numpy as np
import pandas as pd

## $\color{purple}{\text{Setting Up Test Data}}$

$\color{red}{\Large{\text{ ⚠}}}$ We synthensize a statiscally controlled example to more clearly illustrate the concepts. This dataset will satisfy the normality condition set forth in many of the statistical assumptions. These may not carry over to your datasets.

We will cause missingness in approximately 20% of the observations. This may (hopefully) be more that you will experience, but this high proportion will amplify effects such as bias.

`observations` will be the size of our test set. The covariance matrix `cov` supplied shows some nice characteristics with two highly correlated features. But you can generate a completely random covariance matrix using the following:
```
A = np.random.rand(variables, row_size)
cov = np.dot(A, A.transpose()
```
where `variables` is the number of variables and `row_size` is any number greater thanor equal to `variables` to insure a positive semidefinite matrix.

We selected a `mean` to be taken from an normal distribution with a mean between 1 and 5 and a standard deviation between 0 and 5.

This dataset will serve as one of the major datasets for this and subsequent notebooks.

In [39]:
# This covariance matrix has some nice properties to demonstrate. Originally this was generated at random
cov = [
    [1.6545195264181267, 0.6346001403246381, 1.573255077832285, 0.7457615955325402],
    [0.6346001403246381, 0.5636389213610075, 0.5861890592085826, 0.6638139531999303],
    [1.573255077832285, 0.5861890592085826, 1.6461885333121087, 0.4916921086792136],
    [0.7457615955325402, 0.6638139531999303, 0.4916921086792136, 1.0900299890979697],
]
mean = np.random.normal(np.random.uniform(low=1, high=5), np.random.uniform(high=5), 4)

In [40]:
mean

array([-3.44982925,  8.56662185,  4.93010042,  5.06163308])

List the covariance matrix and compare to the original. 
This is only important to insure the number of observations selected is sufficient to give the right characteristics.

In [42]:
observations = 20000
df = pd.DataFrame(
    np.random.multivariate_normal(mean, cov, size=observations),
    columns=["feature a", "feature b", "feature c", "feature d"],
)
df.cov()

,feature a,feature b,feature c,feature d
feature a,1.644890,0.632174,1.559663,0.746702
feature b,0.632174,0.558006,0.585077,0.652994
feature c,1.559663,0.585077,1.626630,0.497506
feature d,0.746702,0.652994,0.497506,1.067495


Now we add one variable that is completely uncorrelated with the other features and show the correlation matrix to confirm.

In [43]:
df["uncorrelated"] = np.random.rand(observations)
df.to_csv('full_set.csv', index=False)

In [44]:
df.corr()

,feature a,feature b,feature c,feature d,uncorrelated
feature a,1.000000,0.659855,0.953494,0.563503,0.007345
feature b,0.659855,1.000000,0.614114,0.846070,0.000318
feature c,0.953494,0.614114,1.000000,0.377547,0.008364
feature d,0.563503,0.846070,0.377547,1.000000,-0.001936
uncorrelated,0.007345,0.000318,0.008364,-0.001936,1.000000


In [45]:
# A function to cause missingness in a given column optionally
def clobber(df, column, probability, depends=[]):
    clob = df[column] == df[column]  # Always True
    for dep_column in depends:
        clob &= df[dep_column] > df[dep_column].median()
    clob *= probability
    rand = np.random.uniform(0, 1, size=len(clob))
    ret = df.copy()  # We copy to avoid clobbering the original
    ret[column] = np.where(clob < rand, df[column], np.nan)
    return ret

In [46]:
def stat_comparison(original, missing, column):
    df = pd.DataFrame.from_dict(
        dict(
            mean=[original[column].mean(), missing[column].mean()],
            median=[original[column].median(), missing[column].median()],
            stdev=[original[column].std(), missing[column].std()],
        ),
        orient="index",
        columns=["Original", "With Missing Data"],
    )
    df["difference"] = (df["Original"] - df["With Missing Data"]).abs()
    df["percentage"] = df["difference"] / df["Original"] * 100
    return df

## $\color{purple}{\text{MCAR and MAR Data Set}}$
In this subsection, we will induce missingness to the dataset we just constructed. This will enable demonstrations of missingness mechanism test as well as to demonstrate treatment techniques in subsequent notebooks

### $\color{purple}{\text{MCAR}}$

In [47]:
mcar_df = clobber(df, "feature a", 0.2)

mcar_df.to_csv('mcar_set.csv', index=False)
mcar_df["feature a"].isnull().sum()

3997

In [48]:
stat_comparison(df, mcar_df, "feature a")

,Original,With Missing Data,difference,percentage
mean,-3.455239,-3.453419,0.001820,-0.052672
median,-3.454605,-3.446362,0.008243,-0.238609
stdev,1.282532,1.286197,0.003665,0.285756


In [51]:
mcar_df.cov()

,feature a,feature b,feature c,feature d,uncorrelated
feature a,1.654304,0.635491,1.570566,0.748323,-0.000240
feature b,0.635491,0.558006,0.585077,0.652994,0.000069
feature c,1.570566,0.585077,1.626630,0.497506,0.003088
feature d,0.748323,0.652994,0.497506,1.067495,-0.000579
uncorrelated,-0.000240,0.000069,0.003088,-0.000579,0.083787


In [50]:
(df.cov() - mcar_df.cov()).abs()/df.cov()*100

,feature a,feature b,feature c,feature d,uncorrelated
feature a,0.572328,5.247330e-01,6.990851e-01,2.170555e-01,1.088015e+02
feature b,0.524733,1.392737e-13,3.036109e-13,6.290760e-13,9.522756e-11
feature c,0.699085,3.036109e-13,2.457106e-13,4.351578e-13,8.005993e-13
feature d,0.217055,6.290760e-13,4.351578e-13,4.368110e-13,-8.089509e-12
uncorrelated,108.801461,9.522756e-11,8.005993e-13,-8.089509e-12,1.656320e-14


In [53]:
double_mcar_df = clobber(mcar_df, "feature b", 0.2)

double_mcar_df.to_csv('double_mcar_set.csv', index=False)
double_mcar_df.isnull().sum()

feature a       3997
feature b       3951
feature c          0
feature d          0
uncorrelated       0
dtype: int64

### $\color{purple}{\text{MAR}}$

In [14]:
mar_df = clobber(df, "feature a", 0.4, depends=["feature c"])

mar_df.to_csv('mar_set.csv', index=False)
mar_df["feature a"].isnull().sum()

3961

In [15]:
stat_comparison(df, mar_df, "feature a")

,Original,With Missing Data,difference,percentage
mean,0.429916,0.194008,0.235908,54.872960
median,0.434693,0.131259,0.303434,69.804284
stdev,1.282751,1.262807,0.019944,1.554770


In [16]:
df.cov()

,feature a,feature b,feature c,feature d,uncorrelated
feature a,1.645449,0.634248,1.571306,0.738141,-0.002366
feature b,0.634248,0.569882,0.589823,0.666099,-0.003302
feature c,1.571306,0.589823,1.650693,0.490209,-0.002004
feature d,0.738141,0.666099,0.490209,1.084064,-0.004531
uncorrelated,-0.002366,-0.003302,-0.002004,-0.004531,0.082964


In [17]:
(df.cov() - mar_df.cov()).abs()/df.cov()*100

,feature a,feature b,feature c,feature d,uncorrelated
feature a,3.085367,2.446220e+00,3.537814e+00,1.150359e+00,-9.626216e+01
feature b,2.446220,1.753346e-13,4.329284e-13,5.166940e-13,-1.050801e-13
feature c,3.537814,4.329284e-13,1.345160e-13,3.397194e-13,-1.254918e-12
feature d,1.150359,5.166940e-13,3.397194e-13,2.048261e-14,-6.700219e-13
uncorrelated,-96.262158,-1.050801e-13,-1.254918e-12,-6.700219e-13,1.672747e-14


In [54]:
double_mar_df = clobber(mar_df, "feature b", 0.4, depends=['feature d'])

double_mar_df.to_csv('double_mar_set.csv', index=False)
double_mar_df.isnull().sum()

feature a       3961
feature b       4045
feature c          0
feature d          0
uncorrelated       0
dtype: int64

## $\color{purple}{\text{Simple MAR Test}}$

The procedure is simple. For any columns with missing data, construct a new column relating to the missingness of that column

In [18]:
test_df = mar_df.copy()

In [19]:
test_df['missingness']=test_df['feature a'].isnull()

Then test to see if that new feature is "related" to any of the columns. If it is then the missingness mechanism is MAR. We will use the "eyeball" test by using correlation. There are statistically robust tests such as using Student's t-test or use logistic regression on the other features to predict the missingness, etc.

In [20]:
test_df.corr()

,feature a,feature b,feature c,feature d,uncorrelated,missingness
feature a,1.000000,0.652763,0.951741,0.555481,-0.012754,NaN
feature b,0.652763,1.000000,0.608130,0.847459,-0.015185,0.230646
feature c,0.951741,0.608130,1.000000,0.366455,-0.005416,0.391072
feature d,0.555481,0.847459,0.366455,1.000000,-0.015108,0.134442
uncorrelated,-0.012754,-0.015185,-0.005416,-0.015108,1.000000,0.009247
missingness,NaN,0.230646,0.391072,0.134442,0.009247,1.000000


In [58]:
test_df = double_mcar_df.copy()

In [59]:
test_df['missingness_a']=test_df['feature a'].isnull()
test_df['missingness_b']=test_df['feature b'].isnull()
test_df.corr()

,feature a,feature b,feature c,feature d,uncorrelated,missingness_a,missingness_b
feature a,1.000000,0.660166,0.953609,0.561643,-0.000647,NaN,0.008199
feature b,0.660166,1.000000,0.614897,0.845913,0.002864,-0.005626,NaN
feature c,0.953609,0.614897,1.000000,0.377547,0.008364,-0.004621,0.007811
feature d,0.561643,0.845913,0.377547,1.000000,-0.001936,-0.002487,0.004325
uncorrelated,-0.000647,0.002864,0.008364,-0.001936,1.000000,0.004491,0.001223
missingness_a,NaN,-0.005626,-0.004621,-0.002487,0.004491,1.000000,0.000437
missingness_b,0.008199,NaN,0.007811,0.004325,0.001223,0.000437,1.000000


## $\color{purple}{\text{Poor Man's Version of Little's MCAR Test (or rather not MAR Test)}}$

The test given above is a little awkward if more than one column has missing data. Originally, Little proposed the following test for MCAR. 

$\color{red}{\text ⚠}$ The code below demonstrates the simplified principle behind Little's MCAR Test but a lot of the statistical rigor has been relaxed.

We adopt the "eyeball" test of whether statistics match or not. In principle, some statistical assumptions are made resulting in a $p$-value. In particular, Little used made normality assumptions resulting in a $\chi^2$ distribution.

First the observations are segregated into their various patterns. In our case, there are only two tests, observation is complete. Observation is missing "feature a"

In [ ]:
pattern1 = mar_df.dropna(subset=["feature a"])
pattern2 = mar_df[mar_df["feature a"].isnull()]

The formal version of Little's Test uses maximum likelihood estimations to estimate statistcal features of each group and compares them. If they are statistcally the same then he declares the missingness mechanism as MCAR. 
Here we use the eyeball test

In [ ]:
pattern1 = mcar_df.dropna(subset=["feature a"])
pattern2 = mcar_df[mcar_df["feature a"].isnull()]

In [ ]:
pd.concat([pattern1.mean(), pattern2.mean()], axis="columns")

In [ ]:
def littles_eyeball_test(df, column):
  pattern1 = df.dropna(subset=[column])
  pattern2 = df[df[column].isnull()]
  return pd.concat([pattern1.mean(), pattern2.mean()], axis="columns")

In [ ]:
littles_eyeball_test(mar_df,'feature a')

In [ ]:
pattern1.drop(columns=["feature a"]).cov()

In [ ]:
pattern2.drop(columns=["feature a"]).cov()

In [ ]:
(
    pattern1.drop(columns=["feature a"]).cov()
    - pattern2.drop(columns=["feature a"]).cov()
).abs()

In [32]:
pattern1 = double_mar_df[double_mar_df['feature a'].isnull() & double_mar_df['feature b'].isnull()]
pattern2 = double_mar_df[double_mar_df['feature a'].isnull() & ~double_mar_df['feature b'].isnull()]
pattern3 = double_mar_df[~double_mar_df['feature a'].isnull() & double_mar_df['feature b'].isnull()]
pattern4 = double_mar_df[~double_mar_df['feature a'].isnull() & ~double_mar_df['feature b'].isnull()]

In [34]:
pattern1.mean()

feature a            NaN
feature b            NaN
feature c       3.654174
feature d       4.515005
uncorrelated    0.507271
dtype: float64

In [36]:
pattern2.mean()

feature a            NaN
feature b      -2.087032
feature c       3.557205
feature d       3.667919
uncorrelated    0.507479
dtype: float64

In [37]:
pattern3.mean()

feature a       0.768768
feature b            NaN
feature c       2.697769
feature d       4.378241
uncorrelated    0.504480
dtype: float64

In [38]:
pattern4.mean()

feature a       0.059974
feature b      -2.539017
feature c       2.230655
feature d       3.311503
uncorrelated    0.499877
dtype: float64

## $\color{purple}{\text{MNAR: The real painful situation}}$


### $\color{purple}{\text{How NOT to synthesize MNAR Missingness}}$

In [ ]:
fmnar_df = clobber(df, "feature a", 0.4, depends=["feature a"])
fmnar_df["feature a"].isnull().sum()

In [ ]:
stat_comparison(df, fmnar_df, "feature a")

In [ ]:
littles_eyeball_test(fmnar_df, 'feature a')

In [ ]:
mnar_df = clobber(df, "uncorrelated", 0.4, depends=["uncorrelated"])

mnar_df.to_csv('mnar_set.csv', index=False)
mnar_df["uncorrelated"].isnull().sum()

In [ ]:
pattern1 = mnar_df.dropna(subset=["uncorrelated"])
pattern2 = mnar_df[mnar_df["uncorrelated"].isnull()]

In [ ]:
littles_eyeball_test(mnar_df, 'uncorrelated')

In [ ]:
pd.concat([pattern1.mean(), pattern2.mean()], axis="columns")

In [ ]:
pattern1.drop(columns=["uncorrelated"]).cov()

In [ ]:
pattern2.drop(columns=["uncorrelated"]).cov()

In [ ]:
(
    pattern1.drop(columns=["uncorrelated"]).cov()
    - pattern2.drop(columns=["uncorrelated"]).cov()
)

In [ ]:
fmnar_df.corr()

In [ ]:
from sklearn.covariance import EmpiricalCovariance

In [ ]:
mcar_df.mean()

In [ ]:
mcar_df.dropna().mean()

In [ ]:
mcar_df[mcar_df["feature a"].isnull()].mean()

In [ ]:
mar_df.dropna().mean()

In [ ]:
mar_df.mean()

In [ ]:
mar_df[mar_df["feature a"].isnull()].mean()

Conclusion on the Theory Section

*   There is no way from just the data itself to distinguish between MCAR and MNAR. 

*   The so-called MCAR tests are really "not MAR" tests
  * Most those tests assume you have already excluded MNAR
* Recommend if the missingness is not MAR assume the worst and treat it as MNAR.
* If missingness is MAR, you should use multivariate imputation not deletion.
* Be careful synthesizing NMAR missingness for benchmarking
